In [1]:
# libraries for EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#To display the plots
%matplotlib inline

# scikit learn library for machine learning algorithms, data preprocessing, and evaluation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, confusion_matrix, log_loss, recall_score, accuracy_score, precision_score, f1_score, roc_auc_score
from sklearn.metrics import make_scorer, average_precision_score, PrecisionRecallDisplay, precision_recall_curve, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer
from sklearn.pipeline import Pipeline

import scikitplot as skplt

# yellowbrick library for visualizing the model performance
from yellowbrick.classifier import ConfusionMatrix, PrecisionRecallCurve
from yellowbrick.classifier import PrecisionRecallCurve
from sklearn.pipeline import Pipeline

# to get rid of the warnings
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

plt.rcParams["figure.figsize"] = (10,6)
sns.set_style("whitegrid", {'axes.grid' : True})

# To display maximum columns
pd.set_option('display.max_columns', 50)

# To display maximum rows
pd.set_option('display.max_rows', 50)

# To set float format
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
def eval_metric(model, X_train, y_train, X_test, y_test):   
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    print("Test_Set")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print()
    print("Train_Set")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

In [30]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_x.csv')

df = train.copy()

In [31]:
df['A'] = df['Yıllık Ortalama Sipariş Verilen Ürün Adedi'] / df['Yıllık Ortalama Sepete Atılan Ürün Adedi']
df['B'] = df['Yıllık Ortalama Satın Alım Miktarı'] / df['Yıllık Ortalama Gelir'] 
df['C'] = df['Yıllık Ortalama Satın Alım Miktarı'] / df['Yıllık Ortalama Sipariş Verilen Ürün Adedi']

In [32]:
test['A'] = test['Yıllık Ortalama Sipariş Verilen Ürün Adedi'] / test['Yıllık Ortalama Sepete Atılan Ürün Adedi'] 
test['B'] = test['Yıllık Ortalama Satın Alım Miktarı'] / test['Yıllık Ortalama Gelir']
test['C'] = test['Yıllık Ortalama Satın Alım Miktarı'] / test['Yıllık Ortalama Sipariş Verilen Ürün Adedi']

In [47]:
df.describe().T.style.background_gradient(axis=1, cmap="Spectral")

,count,mean,std,min,25%,50%,75%,max
index,5460.000000,2729.500000,1576.310566,0.000000,1364.750000,2729.500000,4094.250000,5459.000000
Yıllık Ortalama Gelir,5460.000000,363571.133428,219714.442407,43922.988693,215693.439307,286925.436073,468188.221798,1192437.145258
Yıllık Ortalama Satın Alım Miktarı,5460.000000,16616.612217,14099.171704,2859.254000,4931.859057,8426.818967,30579.244695,48605.594415
Yıllık Ortalama Sipariş Verilen Ürün Adedi,5460.000000,24.040884,14.945655,0.000000,11.550502,20.095924,35.918161,64.616196
Yıllık Ortalama Sepete Atılan Ürün Adedi,5460.000000,73.445693,47.214184,3.977559,25.009168,82.485579,104.473291,242.308441
A,5460.000000,0.421359,0.241901,0.000000,0.217668,0.389002,0.600176,1.000000
B,5460.000000,0.041832,0.024536,0.010590,0.022040,0.033250,0.064344,0.147513
C,5460.000000,inf,nan,109.115531,350.465116,604.526385,1244.863719,inf


In [33]:
from sklearn.model_selection import train_test_split

X = df.drop(["Öbek İsmi", 'index'], axis=1)

y = df["Öbek İsmi"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y)

In [34]:
cat = X_train.select_dtypes("object").columns
cat

Index(['Cinsiyet', 'Yaş Grubu', 'Medeni Durum', 'Eğitim Düzeyi',
       'İstihdam Durumu', 'Yaşadığı Şehir', 'En Çok İlgilendiği Ürün Grubu',
       'Eğitime Devam Etme Durumu'],
      dtype='object')

In [35]:
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', 
                         unknown_value=-1)

column_trans = make_column_transformer((ord_enc, cat), 
                                        remainder='passthrough')

In [36]:
import lightgbm as lgb

operations_lightgbm = [("OrdinalEncoder", column_trans), 
                       ("lightgbm_model", lgb.LGBMClassifier())]

lightgbm_model = Pipeline(steps=operations_lightgbm)

lightgbm_model.fit(X_train, y_train)

Pipeline(steps=[('OrdinalEncoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  Index(['Cinsiyet', 'Yaş Grubu', 'Medeni Durum', 'Eğitim Düzeyi',
       'İstihdam Durumu', 'Yaşadığı Şehir', 'En Çok İlgilendiği Ürün Grubu',
       'Eğitime Devam Etme Durumu'],
      dtype='object'))])),
                ('lightgbm_model', LGBMClassifier())])

In [37]:
eval_metric(lightgbm_model, X_train, y_train, X_test, y_test)

Test_Set
[[ 96   1   2   1   0   2   1   1]
 [  1  80   0   0   0   1   0   0]
 [  0   0  97   0   0   4   1   2]
 [  1   0   1  99   3   0   1   3]
 [  1   0   0   1 104   0   0   0]
 [  1   0   0   2   0  97   2   1]
 [  0   1   1   0   0   0 101   1]
 [  1   0   1   2   0   1   0 103]]
              precision    recall  f1-score   support

      obek_1       0.95      0.92      0.94       104
      obek_2       0.98      0.98      0.98        82
      obek_3       0.95      0.93      0.94       104
      obek_4       0.94      0.92      0.93       108
      obek_5       0.97      0.98      0.98       106
      obek_6       0.92      0.94      0.93       103
      obek_7       0.95      0.97      0.96       104
      obek_8       0.93      0.95      0.94       108

    accuracy                           0.95       819
   macro avg       0.95      0.95      0.95       819
weighted avg       0.95      0.95      0.95       819


Train_Set
[[588   0   0   0   0   0   0   0]
 [  0 465   0

In [26]:
from sklearn.metrics import accuracy_score

y_pred = lightgbm_model.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.9548


In [42]:
import lightgbm as lgb

operations_lightgbm = [("OrdinalEncoder", column_trans), 
                       ("lightgbm_model", lgb.LGBMClassifier(boosting_type='dart',
                                                             colsample_bytree=0.6,
                                                             is_unbalance=True,
                                                             learning_rate=0.01,
                                                             min_child_samples=30,
                                                             n_estimators=1000,
                                                             num_leaves=70,
                                                             reg_alpha=0,
                                                             scale_pos_weight=4,
                                                             subsample_for_bin=3000))]

lightgbm_model = Pipeline(steps=operations_lightgbm)

lightgbm_model.fit(X_train, y_train)

Pipeline(steps=[('OrdinalEncoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  Index(['Cinsiyet', 'Yaş Grubu', 'Medeni Durum', 'Eğitim Düzeyi',
       'İstihdam Durumu', 'Yaşadığı Şehir', 'En Çok İlgilendiği Ürün Grubu',
       'Eğitime Devam Etme Durumu'],
      dtype='object'))])),
                ('lightgbm_model',
                 LGBMClassifier(boosting_type='dart', colsample_bytree=0.6,
                                is_unbalance=True, learning_rate=0.01,
                                min_child_samples=30, n_estimators=1000,
                                num_leaves=70, reg_alpha=0, scale_pos_weight=4,
                                subsample_for_bin=3000))])

In [43]:
eval_metric(lightgbm_model, X_train, y_train, X_test, y_test)

Test_Set
[[ 97   1   1   1   0   2   1   1]
 [  1  80   0   0   0   1   0   0]
 [  0   0  97   0   0   4   1   2]
 [  1   0   1  99   3   0   1   3]
 [  1   0   0   1 104   0   0   0]
 [  1   0   0   2   0  97   2   1]
 [  0   1   1   0   0   0 101   1]
 [  1   0   1   1   0   1   1 103]]
              precision    recall  f1-score   support

      obek_1       0.95      0.93      0.94       104
      obek_2       0.98      0.98      0.98        82
      obek_3       0.96      0.93      0.95       104
      obek_4       0.95      0.92      0.93       108
      obek_5       0.97      0.98      0.98       106
      obek_6       0.92      0.94      0.93       103
      obek_7       0.94      0.97      0.96       104
      obek_8       0.93      0.95      0.94       108

    accuracy                           0.95       819
   macro avg       0.95      0.95      0.95       819
weighted avg       0.95      0.95      0.95       819


Train_Set
[[585   0   1   0   0   1   1   0]
 [  1 464   0

In [29]:
from sklearn.metrics import accuracy_score

y_pred = lightgbm_model.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.9548


In [84]:
predict = lightgbm_model.predict(test)

In [85]:
data = {
    'id': range(2340),
    'Öbek İsmi': predict
}
sub_df = pd.DataFrame(data)
sub_df.set_index('id', inplace=True)
sub_df

,Öbek İsmi
id,
0,obek_3
1,obek_3
2,obek_2
3,obek_6
4,obek_1
...,...
2335,obek_8
2336,obek_2
2337,obek_7


In [86]:
sub_df.to_csv('submission_lgbm_fe6.csv')

In [73]:
import optuna

def objective(trial):
    # LightGBM için hiperparametre aralıklarını belirleme
    params = {
        'objective': 'multiclass',  # Sınıflandırma olduğu için 'binary' olarak ayarlandı
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
    }
    
    operations_lightgbm = [("OrdinalEncoder", column_trans), 
                           ("lightgbm_model", lgb.LGBMClassifier(**params))]

    lightgbm_model = Pipeline(steps=operations_lightgbm)
    # LightGBM modelini oluşturma
    # model = lgb.LGBMClassifier(**params)
    # Modeli veri ile eğitme
    lightgbm_model.fit(X_train, y_train)
    # Doğrulama verisi üzerinde modelin performansını hesaplama (örneğin doğruluk)
    accuracy = lightgbm_model.score(X_test, y_test)
    return accuracy  # Amac fonksiyonu, maximize edilmeye çalışılan değeri döndürmelidir

In [74]:
study = optuna.create_study(direction='maximize')  # Doğruluğu maximize etmeye çalıştığımız için 'maximize'
study.optimize(objective, n_trials=100)  # 100 deneme yaparak hiperparametre optimizasyonu gerçekleştirme

[I 2023-08-24 21:29:55,044] A new study created in memory with name: no-name-64e952ab-9830-4705-bcf5-b560f7311cb1
[I 2023-08-24 21:29:55,438] Trial 0 finished with value: 0.9560439560439561 and parameters: {'num_leaves': 33, 'learning_rate': 0.021406601638270958, 'feature_fraction': 0.9064070445205542}. Best is trial 0 with value: 0.9560439560439561.


[LightGBM] [Warning] feature_fraction is set=0.5686044281958337, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5686044281958337


[I 2023-08-24 21:29:56,298] Trial 1 finished with value: 0.9560439560439561 and parameters: {'num_leaves': 74, 'learning_rate': 0.007201565396984265, 'feature_fraction': 0.5686044281958337}. Best is trial 0 with value: 0.9560439560439561.


[LightGBM] [Warning] feature_fraction is set=0.5959831281472725, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5959831281472725


[I 2023-08-24 21:29:57,098] Trial 2 finished with value: 0.9578754578754579 and parameters: {'num_leaves': 63, 'learning_rate': 0.05357009914065205, 'feature_fraction': 0.5959831281472725}. Best is trial 2 with value: 0.9578754578754579.


...

[I 2023-08-24 21:30:51,448] Trial 97 finished with value: 0.9587912087912088 and parameters: {'num_leaves': 44, 'learning_rate': 0.06638294282792234, 'feature_fraction': 0.579885827912308}. Best is trial 5 with value: 0.9587912087912088.


[LightGBM] [Warning] feature_fraction is set=0.5597608048095633, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5597608048095633


[I 2023-08-24 21:30:52,219] Trial 98 finished with value: 0.9587912087912088 and parameters: {'num_leaves': 79, 'learning_rate': 0.09461942365021009, 'feature_fraction': 0.5597608048095633}. Best is trial 5 with value: 0.9587912087912088.


[LightGBM] [Warning] feature_fraction is set=0.9988582570546629, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9988582570546629


[I 2023-08-24 21:30:53,023] Trial 99 finished with value: 0.9587912087912088 and parameters: {'num_leaves': 61, 'learning_rate': 0.08674991106010442, 'feature_fraction': 0.9988582570546629}. Best is trial 5 with value: 0.9587912087912088.


In [75]:
print('En iyi sonuç (doğruluk): ', study.best_value)
print('En iyi hiperparametreler: ', study.best_params)

En iyi sonuç (doğruluk):  0.9587912087912088
En iyi hiperparametreler:  {'num_leaves': 38, 'learning_rate': 0.04344729836905109, 'feature_fraction': 0.5272523507226217}
